<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>

### 1. `pipelines.py`의 역할

1. **데이터 정제(Data Cleaning)**
   - 크롤링된 데이터에서 불필요한 공백 제거, 문자열 형식 통일, 날짜 형식 변환 등의 작업을 수행합니다.

2. **데이터 검증(Data Validation)**
   - 필수 필드의 존재 여부 확인, 데이터 형식 검증, 값의 범위 검사 등을 통해 데이터의 정확성을 보장합니다.

3. **중복 데이터 제거(Deduplication)**
   - 고유성을 보장해야 하는 데이터에 대해 중복 검사를 수행하고, 중복된 데이터는 제거합니다.

4. **데이터 후처리(Post-processing)**
   - 데이터가 최종적으로 다른 시스템으로 전달되기 전에 추가적인 후처리 작업을 수행합니다.

### 2. 관련 문법 및 설정

`pipelines.py`에서 파이프라인을 구현하기 위한 기본적인 클래스 구조와 메소드, Scrapy 프로젝트 설정 방법은 다음과 같습니다.

1. **파이프라인 클래스 정의**
   - 각 파이프라인은 하나의 Python 클래스로 정의되며, `process_item` 메소드를 반드시 구현해야 합니다.
   ```python
   from scrapy.exceptions import DropItem

   class MyPipeline:
       def process_item(self, item, spider):
           # 아이템 처리 로직
           return item
   ```

2. **`process_item` 메소드**
   - 이 메소드는 각 아이템이 파이프라인을 통과할 때 호출됩니다.
   - 아이템을 수정하거나, 유효하지 않은 아이템을 제거하기 위해 `DropItem` 예외를 발생시킬 수 있습니다.
   - 모든 파이프라인을 통과한 아이템은 최종적으로 Scrapy 엔진에 의해 처리됩니다.

3. **파이프라인 활성화 및 우선순위 설정**
   - `settings.py` 파일에서 `ITEM_PIPELINES` 설정을 통해 사용할 파이프라인을 활성화하고, 실행 순서(우선순위)를 지정합니다.
   ```python
   ITEM_PIPELINES = {
       'myproject.pipelines.MyPipeline': 300,
   }
   ```
   - 숫자가 낮을수록 먼저 실행됩니다.




### 3. Scrapy 프로젝트2

> 본 프로젝트는 크롤링 후 데이터 정제 로직을 `pipelines.py`에 구현하는 과정을 단계별로 설명

1. **프로젝트 생성**
   - 터미널을 열고 아래 명령어를 실행하여 Scrapy 프로젝트를 생성합니다.
   ```bash
   scrapy startproject daveleefun_project2
   ```
   이 명령은 `daveleefun_project2`라는 이름의 새 Scrapy 프로젝트를 만듭니다.

2. **프로젝트 디렉토리로 이동**
   - 생성된 프로젝트 폴더로 이동합니다.
   ```bash
   cd daveleefun_project2
   ```

3. **Spider 생성**
   - 프로젝트에 크롤링할 대상 사이트를 지정하여 Spider를 생성합니다.
   ```bash
   scrapy genspider daveleefun2 davelee-fun.github.io
   ```
   `daveleefun`은 생성할 Spider의 이름이고, `davelee-fun.github.io`는 크롤링할 사이트의 주소입니다.

### 4. 아이템 정의 및 Spider 수정

1. **`items.py` 파일 수정**
   - `items.py` 파일에서 크롤링할 데이터의 구조를 정의합니다.
   ```python
   import scrapy

   # 크롤링된 아이템을 위한 클래스 정의
   class DaveleefunItem(scrapy.Item):
       title = scrapy.Field()  # 게시글의 제목
       description = scrapy.Field()  # 게시글의 설명
   ```

2. **Spider 수정 (`daveleefun2.py`)**
   - Spider 파일을 수정하여 크롤링 로직을 구현합니다.
   ```python
   import scrapy
   from daveleefun_project2.items import DaveleefunItem  # 정의한 아이템 클래스 임포트

   class DaveleefunSpider(scrapy.Spider):
       name = 'daveleefun2'
       allowed_domains = ['davelee-fun.github.io']
       start_urls = ['https://davelee-fun.github.io/']

       def parse(self, response):
           # 크롤링된 데이터를 아이템 객체에 저장
           item = DaveleefunItem()
           item['title'] = response.css('h1.sitetitle::text').get()
           item['description'] = response.xpath('//p[@class="lead"]/text()').get()
           yield item  # 아이템 반환
   ```
   이 코드에서 `parse` 메소드는 크롤링할 웹 페이지로부터 데이터를 추출하고, `DaveleefunItem` 객체에 저장하여 반환합니다.

### 5. `pipelines.py`에서 데이터 정제 로직 추가

1. **`pipelines.py` 파일 수정**
   - 데이터 정제를 위한 파이프라인을 `pipelines.py`에 추가합니다.
   ```python
   from scrapy.exceptions import DropItem

   # description 필드의 공백을 제거하는 파이프라인
   class DescriptionPipeline:
       def process_item(self, item, spider):
           if item['description']:
               item['description'] = item['description'].strip()  # 공백 제거
               return item
           else:
               raise DropItem("Missing description in %s" % item)
   ```

2. **`settings.py`에서 파이프라인 활성화**
   - `settings.py` 파일을 수정하여 `DescriptionPipeline`을 활성화하고 우선순위를 설정합니다.
   ```python
   ITEM_PIPELINES = {
      'daveleefun_project2.pipelines.DescriptionPipeline': 300,  # 파이프라인 활성화 및 우선순위 설정
   }
   ```

### 6. 크롤링 실행 및 결과 확인
   - 터미널에서 아래 명령어로 크롤링을 실행합니다.
   ```bash
   scrapy crawl daveleefun2 -o results.json
   ```

### 7. Scrapy 프로젝트3

1. **프로젝트 생성**
   - 새 Scrapy 프로젝트를 생성합니다.
   ```bash
   scrapy startproject daveleefun_project3
   ```
   이 명령어로 `daveleefun_project3`라는 이름의 새 Scrapy 프로젝트가 생성됩니다.

2. **프로젝트 디렉토리로 이동**
   ```bash
   cd daveleefun_project3
   ```

3. **Spider 생성**
   - `davelee-fun.github.io` 사이트를 크롤링할 Spider를 생성합니다.
   ```bash
   scrapy genspider category_spider davelee-fun.github.io
   ```

### 8. 아이템 및 Spider 수정

1. **`items.py` 파일 수정**
   - 크롤링할 데이터 구조를 `CategoryItem` 클래스로 정의합니다.
   ```python
   import scrapy

   class CategoryItem(scrapy.Item):
       category = scrapy.Field()  # 카테고리 필드 정의
   ```

2. **Spider 수정 (`category_spider.py`)**
   - 크롤링 로직에서 `category` 필드를 추출하고 반환하도록 수정합니다.
```python
import scrapy
from daveleefun_project3.items import CategoryItem

class CategorySpider(scrapy.Spider):
    name = 'category_spider'
    start_urls = ['https://davelee-fun.github.io']

    def parse(self, response):
        categories = response.css('a.text-dark::text').getall()
        for category in categories:
            item = CategoryItem()
            item['category'] = category
            yield item  # yield 구문을 for 반복문 안으로 이동

```
   이 코드는 웹 페이지로부터 `category` 데이터를 추출하여 `CategoryItem` 객체에 저장하고 반환합니다.

### 9. `pipelines.py`에서 중복 제거 및 문자열 후처리 설정

1. **중복 제거 및 문자열 후처리 로직 추가**
   - 데이터 처리를 위해 `SetPipeline`과 `RemovePhrasePipeline`을 정의합니다.
```python
from scrapy.exceptions import DropItem

class CleanCategoryPipeline:
    def process_item(self, item, spider):
        item['category'] = item['category'].strip()
        return item

class SetPipeline:
    def __init__(self):
        self.categories_seen = set()

    def process_item(self, item, spider):
        if item['category'] in self.categories_seen:
            raise DropItem("Duplicate item found: %s" % item)
        else:
            self.categories_seen.add(item['category'])
            return item

class RemovePhrasePipeline:
    def process_item(self, item, spider):
        item['category'] = item['category'].replace(" 관련 상품 추천", "")
        return item

```

2. **`settings.py`에서 파이프라인 활성화**
   - 두 파이프라인을 활성화하고 우선순위를 설정
   - ITEM_PIPELINES 설정에서 지정한 숫자는 실행 순서를 결정
   - 파이프라인의 우선순위를 나타내며, 숫자가 낮을수록 먼저 실행
   - 우선순위가 낮다고 해서 파이프라인이 실행되지 않는 것은 아님 (모든 지정된 파이프라인은 Scrapy에 의해 실행)
   - 따라서, SetPipeline이 RemovePhrasePipeline보다 먼저 실행되어 아이템에서 중복을 제거한 후, 
   - 그 다음으로 RemovePhrasePipeline에서 아이템의 특정 문자열을 제거하는 과정이 진행

```python
ITEM_PIPELINES = {
    "daveleefun_project3.pipelines.SetPipeline": 100,
    "daveleefun_project3.pipelines.CleanCategoryPipeline": 200,
    "daveleefun_project3.pipelines.RemovePhrasePipeline": 300,
}
```

### 10. 크롤링 실행 및 결과 확인

1. **크롤링 실행**
   - 터미널에서 아래 명령어를 사용하여 크롤링을 실행하고, 결과를 `categories.json`에 저장합니다.
   ```bash
   scrapy crawl category_spider -o categories.json
   ```

2. **결과 확인**
   - `categories.json` 파일을 열어 중복된 `category` 값이 제거되었고, "관련 상품 추천" 문자열이 제거된 결과를 확인합니다.

<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>